In [107]:
import pandas as pd

df = pd.read_csv("../../../iPinyou/make-ipinyou-data-master/3476/train.log.txt",header=0,sep='\t',index_col=False)

In [108]:
print df.shape

(1970360, 27)


In [109]:
print df[["click","weekday","hour","advertiser"]].head()

   click  weekday  hour  advertiser
0      0        6     0        3476
1      0        6     0        3476
2      0        6     0        3476
3      0        6     0        3476
4      0        6     0        3476


In [110]:
###############Campaign cvr
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np


df_sub_cvr =  df[["click","weekday","hour","advertiser"]]
df_add_features = pd.DataFrame(df_sub_cvr.groupby(['advertiser','weekday','hour','click']).size().reset_index(name='count'))
print df_add_features.head()

   advertiser  weekday  hour  click  count
0        3476        0     0      0  14134
1        3476        0     0      1      6
2        3476        0     1      0   6301
3        3476        0     1      1      3
4        3476        0     2      0   5261


In [111]:
#i = 0
for name, group in df_add_features.groupby(['advertiser','weekday','hour']): 
    #i += 1
    #print name[1]

    counts = group['count'].values
    #print group
    #print group['purchase'].values
    
    if ((len(counts) == 1) & (group['click'].values[0] == 0)):
        cvr_h = 0
    elif ((len(counts) == 1) & (group['click'].values[0] == 1)):
        cvr_h = 1
    else:
        cvr_h = float(counts[1])/float(counts.sum())
    
    df.ix[(df.advertiser==name[0]) & (df.weekday == name[1])
              & (df.hour == name[2]), 'cvr_h'] = cvr_h

    #if i == 10:
    #    break

In [112]:
# compare the previous two hours (t-1, t-2) cvr
from itertools import izip
from math import log
import sys
print df.shape[0]

cvr_diff_df = df.drop_duplicates(['advertiser','weekday','hour'])[['advertiser','weekday','hour','cvr_h']]
print cvr_diff_df.shape[0]
print cvr_diff_df.head()

  
epsilon = np.finfo(float).eps

def myfunc(group):
    group = group.sort_values(['weekday', 'hour'], ascending=[True, True])
    for i in range(1,2):
        diff_cvr = []
        diff_cvr.extend(np.zeros(i+1,dtype=np.int))
        cvr_col = group['cvr_h'].values
             
        for x, y in zip(cvr_col,cvr_col[1:]):
            if y - x > 0.0:
                diff_cvr.append(1)
            elif y - x == 0.0:
                diff_cvr.append(0)
            else: 
                diff_cvr.append(-1)
                
        col_name = 'diff_cvr_' + str(i)
        group[col_name] = pd.Series(diff_cvr[:-i], index=group.index)
        #print diff_cvr
        #group['cvr_ratio'] = pd.Series(ratio_cvr[:-1], index=group.index)
    #print group
    return group


cvr_diff_result = cvr_diff_df.groupby(['advertiser']).apply(myfunc)
print cvr_diff_result.tail()
#print cvr_diff_result[cvr_diff_result['cvr_ratio'] != 1].head()
#print cvr_diff_result[cvr_diff_result['idcampaign'] == 90]

1970360
142
       advertiser  weekday  hour     cvr_h
0            3476        6     0  0.000588
13613        3476        6     1  0.000467
22186        3476        6     2  0.000322
28398        3476        6     3  0.000488
34549        3476        6     4  0.000000
                   advertiser  weekday  hour     cvr_h  diff_cvr_1
advertiser                                                        
3476       262957        3476        6    19  0.000772           1
           283673        3476        6    20  0.000757           1
           304814        3476        6    21  0.000468          -1
           326188        3476        6    22  0.000592          -1
           349836        3476        6    23  0.000219           1


In [113]:
df_merge = pd.merge(df,cvr_diff_result[['advertiser','weekday','hour','diff_cvr_1']], 
                    how='left', on=['advertiser','weekday','hour'])
print df_merge.tail()
print df_merge.shape[0]

         click  weekday  hour                             bidid  \
1970355      0        5    10  da46db053c30d4a2b25626dbeb3cef3d   
1970356      0        5    10  7934703736c4d3bb957419581ed0b0a4   
1970357      0        5    10  d9216cb947b77ad6ad8a5bc75d0075e8   
1970358      0        5    10  cf88d17dc911ad1841094ce8348eed0a   
1970359      0        5    10  8f7abf2b1ea50caef7e0969350f7d375   

                 timestamp  logtype        ipinyouid       useragent  \
1970355  20130612103253642        1  VhdVLidpDq1ZaVb  windows_chrome   
1970356  20130612103255375        1  Vh5zC35lDNj4Xhc      windows_ie   
1970357  20130612103302190        1  VhkrOvlIL2dvDOE      windows_ie   
1970358  20130612103318102        1  Vh1HOwmPD9CsQQ2      windows_ie   
1970359  20130612103318168        1  Vh1HOwmPD9CsQQ2      windows_ie   

                   IP  region     ...      slotformat  slotprice  \
1970355  112.231.25.*     146     ...               1          0   
1970356   222.40.31.*     16

In [114]:
df_merge.to_csv('../../../iPinyou/ipinyou-data-addflag/3476.train.flag.txt',index=False, sep="\t")